In [1]:
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd

from tqdm import tqdm

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
cheques = pd.read_csv('/kaggle/input/final-hack4retail/cheques_public.csv', sep=';')
darkstore_map = pd.read_csv('/kaggle/input/final-hack4retail/darkstore_map.csv', sep=';')
distances = pd.read_csv('/kaggle/input/final-hack4retail/darkstore.csv').values

# baseline_submit = pd.read_csv('/kaggle/input/final-hack4retail/baseline_submit.csv')
baseline_submit = pd.read_csv('/kaggle/input/final-hack4retail/optuna_6_features.csv')
strange_submit = pd.read_csv('/kaggle/input/hack4retail-genetic/sub.csv')

In [3]:
def validator(_cheques, _distances, darkstore_map):
    darkstore_map_dict = darkstore_map.set_index(['SECTION', 'LEVEL']).to_dict('index')
    darkstore_map_dict_inverse = dict()
    for k, v in darkstore_map_dict.items():
        darkstore_map_dict[k] = v['LAGERID']
        darkstore_map_dict_inverse[v['LAGERID']] = k
        
    picking_time = {1: 1, 2: 2, 3: 3}
        
    cheques = _cheques.copy()
    cheques['LOCATION'] = cheques['LAGERID'].apply(lambda x: darkstore_map_dict_inverse[x])
    cheques[['SECTION', 'LEVEL']] = pd.DataFrame(cheques['LOCATION'].tolist(), index=cheques.index)
    
    all_times = []
    for cheque, temp_cheque in cheques.groupby('CHEQUEID'):

        sum_time = 0
        current_location = 0
        sum_time += (temp_cheque['KOLVO'] * temp_cheque['LEVEL'].map(picking_time)).sum()
        
        est_locatsii = True
        set_locations = set(temp_cheque['SECTION'])
        while est_locatsii:
            dists = sorted([(x, _distances[current_location, x]) for x in set_locations], key=lambda x: x[1], reverse=False)
            current_location = dists[0][0]
            travel_time = dists[0][1] * 2
            sum_time += travel_time

            set_locations.remove(current_location)
            if not len(set_locations):
                est_locatsii = False

        dist_to_final = _distances[0, current_location]
        sum_time += dist_to_final * 2
        
        
        all_times.append(sum_time)
        
    return np.sum(all_times), all_times

In [17]:
validator(cheques, distances, baseline_submit)[0]

In [12]:
from random import choice
from joblib import Parallel, delayed
from time import time


class GeneticDarkstore:
    def __init__(self, item_count, mutation_percent, init_population_count=30, threshold=0.5):
        self.towns_count = item_count
        self.mutation_chance = mutation_percent / 100
        self.init_population_count = init_population_count
        self.init_population = []
        for i in range(self.init_population_count):
            self.init_population.append(list(
                np.random.permutation(range(1, self.towns_count+1))
            ))
        self.population = self.init_population.copy()
        self.iteration = 0
        self.threshold = threshold

    def getRouteDist(self, route):
        temp_df = baseline_submit.copy()
        temp_df['LAGERID'] = route
        res, _ = validator(cheques, distances, temp_df)
        return res

    def getAllDist(self):
        distances = Parallel(n_jobs=-1)(
            delayed(self.getRouteDist)(route) for route in self.population
        )
        return distances

    def getMinDist(self):
        return min(self.getAllDist())

    def getBestRoute(self):
        return self.population[np.argmin(self.getAllDist())]

    def makeCrossover(self, parent):
        n = np.random.choice(len(parent)-1)
        child = np.append(parent[:n], np.roll(parent[n:], 1))
        return child.astype(int)

    def makeMutation(self, child):
        positions = np.random.choice(len(child), size=2)
        while True:
            child[positions[0]], child[positions[1]] = child[positions[1]], child[positions[0]]
            positions[0] += 1
            positions[1] -= 1
            if positions[0] >= positions[1]:
                break
            

    def makeStep(self, verbose=True):
        new_population = []
        # отбор родителей, выбираем 100 * threshold % лучших по критерию наименьшей длины маршрута
        parents = np.argsort(self.getAllDist())
        parents = parents[:int(len(parents)*self.threshold)]
        # скрещивание
        while len(new_population) < self.init_population_count:
            current_parent = np.random.choice(parents)
            new_population.append(self.makeCrossover(self.population[current_parent]))

        # делаем мутации
        for child in new_population:
            if np.random.choice([0, 1], p=[1-self.mutation_chance, self.mutation_chance]):
                self.makeMutation(child)
                
        self.population = new_population
        self.iteration += 1
        if verbose:
            print(self.iteration, self.getMinDist()) # для отладки

    def makeCrossoverAdvanced(self, parent1, parent2):
        # код из текста лабы
        def buildCycles(p1, p2, index):
            cycle1, cycle2 = {}, {}
            while index not in cycle1:
                cycle1[index] = p1[index]
                p2_val = p2[index]
                cycle2[index] = p2_val
                index = p1.index(p2_val)
            return cycle1, cycle2
        cycles_set1, cycles_set2 = [], []
        for index in range(len(parent1)):
            if any([index in cycle for cycle in cycles_set1]): continue
            cycle1, cycle2 = buildCycles(parent1, parent2, index)
            cycles_set1.append(cycle1)
            cycles_set2.append(cycle2)
        child = [None] * len(parent1)
        for c1, c2 in zip(cycles_set1, cycles_set2):
            c = np.random.choice([c1, c2])
            for key in c:
                child[key] = c[key]
        return child

    def makeStepAdvanced(self, verbose=True):
        new_population = []
        # собираем детей всех пар родителей
        for i in range(self.init_population_count):
            for j in range(i):
                new_population.append(
                    self.makeCrossoverAdvanced(
                        self.population[i], self.population[j]
                    )
                )

        # отбираем лучших детей
        print('\tgathering children...', end=' ')
        child_dist = Parallel(n_jobs=-1)(
            delayed(
                (lambda child: (child, self.getRouteDist(child))))(child)
                for child in new_population
            )
        child_dist.sort(key=lambda t: t[1])
        new_population = [t[0] for t in child_dist]
        new_population = new_population[:self.init_population_count]
        print('\tDone!')

        # делаем мутации
        print('\tmaking mutations...', end=' ')
        for child in new_population:
            if np.random.choice([0, 1], p=[1-self.mutation_chance, self.mutation_chance]):
                self.makeMutation(child)
        print('\tDone!')
                
        self.population = new_population
        self.iteration += 1
        if verbose:
            print('SCORE:\t', self.getMinDist()) # для отладки

In [13]:
solver = GeneticDarkstore(
    item_count=len(darkstore_map), 
    mutation_percent=50, 
    init_population_count=20, 
    threshold=0.5
)

In [14]:
base_population = list(baseline_submit['LAGERID'].values)
new_init_population = [base_population]
for i in range(solver.init_population_count-1):
    temp = base_population.copy()
    solver.makeMutation(temp)
    new_init_population.append(temp)
    
solver.init_population = new_init_population
solver.population = new_init_population

In [16]:
solver.getMinDist()

In [18]:
iter_count = 10

for i in range(iter_count):
    print(f'ITER:\t {i}')
    tick = time()
    solver.makeStepAdvanced()
    res = solver.getBestRoute()
    tock = time()
    print(f'TIME:\t {(tock - tick):.3f} s')
    sub = darkstore_map.copy()
    sub['LAGERID'] = res
    sub.to_csv('sub.csv', index=False)
    print('\n')

In [19]:
strange_submit = pd.read_csv('sub.csv')

strange_submit['LEVEL'] = baseline_submit['LEVEL'] 
strange_submit['SECTION'] = baseline_submit['SECTION'] 

strange_submit.sort_values(by=['SECTION', 'LEVEL'], inplace=True)


times_sum, all_times = validator(cheques, distances, strange_submit)
times_sum

In [21]:
strange_submit.to_csv('sub_final.csv', index=False)